# MultiQueryRetriever
- The MultiQueryRetriever automates prompt tuning by generating multiple queries for a user input query and combines the results

- https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever

In [1]:
import os
from getpass import getpass
from langchain import HuggingFaceHub

In [2]:
HUGGINGFACEHUB_API_TOKEN = getpass()

········


In [3]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [4]:
llm = HuggingFaceHub(repo_id= 'mistralai/Mixtral-8x7B-Instruct-v0.1',
                           model_kwargs={'temperature': 0.5, 'max_length': 100})

In [5]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
# Load blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

In [8]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

In [9]:
# VectorDB
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [10]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What are the approaches to Task Decomposition?"
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [11]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [12]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['You are an AI language model assistant. Your task is ', '    to generate 3 different versions of the given user ', '    question to retrieve relevant documents from a vector  database. ', '    By generating multiple perspectives on the user question, ', '    your goal is to help the user overcome some of the limitations ', '    of distance-based similarity search. Provide these alternative ', '    questions separated by newlines. Original question: What are the approaches to Task Decomposition?', '', 'Version 1: Can you provide some techniques for breaking down tasks into smaller components?', 'Version 2: How do people usually approach the problem of dividing complex tasks into manageable parts?', 'Version 3: What are some strategies for decomposing tasks into simpler, more manageable pieces?']


29